# Reserve capacity allocation

The model setup for the reserve capacity examples presented below are available in the following formats.

- pyshop
    - [reserve_cap.py](#reserve_cap.py)
- YAML
    - [model.yaml](#model.yaml)
    - [reserve_obligation.yaml](#reserve_obligation.yaml)
    - [smooth_reserve.yaml](#smooth_reserve.yaml)
- ASCII
    - [model.ascii](#model.ascii)
    - [reserve_obligation.ascii](#reserve_obligation.ascii)
    - [smooth_reserve.ascii](#smooth_reserve.ascii)

## Optimal distribution of reserve capacity
A simple system with 6 generators located on two separate plants will be used to illustrate the basic features of the **reserve_group** object and the reserve functionality in SHOP. First, we create and run the basic model without any reserve requirements.

In [15]:
#Necessary imports used in all examples
import pandas as pd
import cufflinks as cf
import plotly.offline as py
import plotly.graph_objs as go
cf.go_offline()
py.offline.init_notebook_mode(connected=True)
from pyshop import ShopSession

#Functions used in this example for building and solving a simple model with cuts
from reserve_cap import build_model, run_model

In [16]:
#Create a standard ShopSession
shop=ShopSession()
#Build a simple model with two reservoirs, two plants, and 6 generators.
build_model(shop)
#Display topology to the screen
display(shop.model.build_connection_tree())

#Run an optimization without any reserve obligations
run_model(shop)
#Display the optimal production level
pd.DataFrame([gen.production.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="Production without reserve requirement", kind='bar', barmode = 'stack')

Now we create an identical SHOP model but add two **reserve_group** objects with some specified FCR-N and FRR reserve obligations. All generators are part of the FCR-N group and can help cover the given **fcr_n_up_obligation** and **fcr_n_down_obligation**. Only the generators in Plant2 are part of the FRR group and have to cover the **frr_up_obligation** without help from the other generators in Plant1. 

In [17]:
#Create a new shop session
shop=ShopSession()
build_model(shop)

#Add two reserve_group objects to the original model
fcr_n = shop.model.reserve_group.add_object("fcr_n_group")
fcr_n.fcr_n_up_obligation.set(10)
fcr_n.fcr_n_down_obligation.set(10)

frr = shop.model.reserve_group.add_object("frr_group")
frr.frr_up_obligation.set(15)

#Connect all generators to the fcr_n group
for gen in shop.model.generator:
    gen.connect_to(fcr_n)

#Connect only the generators in Plant2 to the frr group
plant2 = shop.model.plant.Plant2    
for gen in plant2.generators:
    gen.connect_to(frr)    
    
#Optimize model
run_model(shop) 

#Plot the resulting optimized production schedules
pd.DataFrame([gen.production.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="Production with reserve requirement", kind='bar', barmode = 'stack')

The optimal production set points have shifted for some of the generators due to the new reserve capacity obligations. The plots below show how the reserve capacity has been distributed among the generators.

In [18]:
pd.DataFrame([gen.frr_up_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FRR up delivery", kind='bar', barmode = 'stack')
pd.DataFrame([gen.fcr_n_up_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FCR-N up delivery", kind='bar', barmode = 'stack')
pd.DataFrame([gen.fcr_n_down_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FCR-N down delivery", kind='bar', barmode = 'stack')

Note that the FCR-N reserve capacity is not always symmetric since no **fcr_n_equality_flag** on the global_settings object (or equivalent command) has been specified. 

The following plots give a more detailed look at the individual generator production and reserve capacity results:

In [19]:
for gen in shop.model.generator:
    name = gen.get_name()
    p_min = gen.p_min.get()
    p_max = gen.p_max.get()
    prod = gen.production.get()
    frr_up = gen.frr_up_delivery.get()
    fcr_n_up = gen.fcr_n_up_delivery.get()
    fcr_n_down = gen.fcr_n_down_delivery.get()

    t = prod.index
    
    fig = go.Figure(layout={'title':"Production and reserves: "+name,'xaxis_title':"Time",'yaxis_title':"Production and reserves [MW]"})
    
    fig.add_trace(go.Scatter(name="P_min",x=t,y=[p_min]*len(t),line={'color': "black", 'width': 1,'dash':"dash"}))    
    fig.add_trace(go.Scatter(name="P_max",x=t,y=[p_max]*len(t),line={'color': "black", 'width': 1,'dash':"dash"}))    
    fig.add_trace(go.Scatter(name="Production",x=t,y=prod.values,line={'color': "black", 'width': 1},line_shape='hv'))    

    fig.add_trace(go.Scatter(showlegend=False,x=t,y=prod.values-fcr_n_down,line={'color': "black", 'width': 0},line_shape='hv'))    
    fig.add_trace(go.Scatter(name="FCR-N down",x=t, y=prod,fill='tonexty',line={'color': "orange", 'width': 0},line_shape='hv'))
    fig.add_trace(go.Scatter(name="FCR-N up",x=t,y=prod+fcr_n_up, fill='tonexty',line={'color': "red", 'width': 0},line_shape='hv'))    
    fig.add_trace(go.Scatter(name="FRR up",x=t,y=prod+frr_up+fcr_n_up, fill='tonexty',line={'color': "blue", 'width': 0},line_shape='hv'))    
    
    fig.show()


It is possible that the sum of the production and the upward reserve capacity is slightly higher than the maximum production limit. This is usually due to the deviation between optimized and post-calculated production in SHOP, reported as **prod_unbalance** on the plant object. Non-linearities due to head loss effects and the turbine and generator efficiency curves are linearized and iteratively refined in SHOP. The optimized production variables are therefore approximations of the non-linear production function. A small gap is likely to appear when the optimized value is compared to a true non-linear post-calculation of the production based on the optimized discharge. The production unbalance means that the reserve capacity constraints are not broken in the optimization problem even though they appear to be broken when using the post-calculated production reported from SHOP. These are some of the limitations of a linear optimization model.

The amount of reserve capacity allocated on each unit is not very stable over the optimization horizon in the previous optimization run. A way to rectify this is to add a **gen_reserve_ramping_cost** on the global_settings object, penalizing any change in allocation of reserve capacity between two time steps. The example balow adds a reserve ramping cost of 5 €/MW. In addition, a minimum limit of 1 MW is set for delivering the three different reserve capacity products. The potential downside of adding a reserve capacity ramping cost and minimum limits is the increased calculation time.

In [20]:
#Create a new shop session
shop=ShopSession()
build_model(shop)

#Add two reserve_group objects to the original model
fcr_n = shop.model.reserve_group.add_object("fcr_n_group")
fcr_n.fcr_n_up_obligation.set(10)
fcr_n.fcr_n_down_obligation.set(10)

frr = shop.model.reserve_group.add_object("frr_group")
frr.frr_up_obligation.set(15)

#Connect all generators to the fcr_n group
for gen in shop.model.generator:
    gen.connect_to(fcr_n)
    
    gen.fcr_n_up_min.set(1)
    gen.fcr_n_down_min.set(1)
    
#Connect only the generators in Plant2 to the frr group
plant2 = shop.model.plant.Plant2    
for gen in plant2.generators:
    gen.connect_to(frr)    
    
    gen.frr_up_min.set(1)
    
#Add a reserve ramping cost    
settings = shop.model.global_settings.global_settings
settings.gen_reserve_ramping_cost.set(5)
    
#Optimize model
run_model(shop) 

#Plot the reserve capacity distribution
pd.DataFrame([gen.frr_up_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FRR up delivery", kind='bar', barmode = 'stack')
pd.DataFrame([gen.fcr_n_up_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FCR-N up delivery", kind='bar', barmode = 'stack')
pd.DataFrame([gen.fcr_n_down_delivery.get().rename(gen.get_name()) for gen in shop.model.generator]).transpose().iplot(title="FCR-N down delivery", kind='bar', barmode = 'stack')


Generator 1 on Plant2 delivers most of the reserve capacity when minimum limits and reserve ramping costs are added to the optimization problem. The unit commitment and production schedules of the generators have been altered to accomodoate the smoother reserve capacity commitment:

In [21]:
for gen in shop.model.generator:
    name = gen.get_name()
    p_min = gen.p_min.get()
    p_max = gen.p_max.get()
    prod = gen.production.get()
    frr_up = gen.frr_up_delivery.get()
    fcr_n_up = gen.fcr_n_up_delivery.get()
    fcr_n_down = gen.fcr_n_down_delivery.get()

    t = prod.index
    
    fig = go.Figure(layout={'title':"Production and reserves: "+name,'xaxis_title':"Time",'yaxis_title':"Production and reserves [MW]"})
    
    fig.add_trace(go.Scatter(name="P_min",x=t,y=[p_min]*len(t),line={'color': "black", 'width': 1,'dash':"dash"}))    
    fig.add_trace(go.Scatter(name="P_max",x=t,y=[p_max]*len(t),line={'color': "black", 'width': 1,'dash':"dash"}))    
    fig.add_trace(go.Scatter(name="Production",x=t,y=prod.values,line={'color': "black", 'width': 1},line_shape='hv'))    

    fig.add_trace(go.Scatter(showlegend=False,x=t,y=prod.values-fcr_n_down,line={'color': "black", 'width': 0},line_shape='hv'))    
    fig.add_trace(go.Scatter(name="FCR-N down",x=t, y=prod,fill='tonexty',line={'color': "orange", 'width': 0},line_shape='hv'))
    fig.add_trace(go.Scatter(name="FCR-N up",x=t,y=prod+fcr_n_up, fill='tonexty',line={'color': "red", 'width': 0},line_shape='hv'))    
    fig.add_trace(go.Scatter(name="FRR up",x=t,y=prod+frr_up+fcr_n_up, fill='tonexty',line={'color': "blue", 'width': 0},line_shape='hv'))    
    
    fig.show()

## reserve_cap.py <a name="reserve_cap.py"></a>

In [22]:
%pycat reserve_cap.py

import pandas as pd
import numpy as np

def build_model(shop):
    starttime = pd.Timestamp('2018-01-23 00:00:00')
    endtime = pd.Timestamp('2018-01-26')
    shop.set_time_resolution(starttime=starttime, endtime=endtime, timeunit="hour", timeresolution=pd.Series(index=[starttime],data=[1]))
    
    rsv1 = shop.model.reservoir.add_object('Reservoir1')
    rsv1.max_vol.set(39)
    rsv1.lrl.set(860)
    rsv1.hrl.set(905)
    rsv1.vol_head.set(pd.Series([860, 906, 907], index=[0, 39, 41.66], name=0))    

    rsv2 = shop.model.reservoir.add_object('Reservoir2')
    rsv2.max_vol.set(97.5)   
    rsv2.lrl.set(650)   
    rsv2.hrl.set(679)    
    rsv2.vol_head.set(pd.Series([650, 679, 680], index=[0, 97.5, 104.15], name=0))
    
    plant1 = shop.model.plant.add_object('Plant1')
    plant1.outlet_line.set(672)
    plant1.main_loss.set([0])
    plant1.penstock_loss.set([0.001])
    plant1.mip_flag.set(1)
    for gen_no in range(2):
        gen=shop.model.generator.add_object(f"{plant1.get_

## model.yaml <a name="model.yaml"></a>

In [23]:
%pycat model.yaml

time:
  starttime: 2018-01-23 00:00:00
  endtime: 2018-01-26 00:00:00
  timeunit: hour
  timeresolution:
    2018-01-23 00:00:00: 1
model:
  reservoir:
    Reservoir1:
      start_head: 900
      max_vol: 39
      lrl: 860
      hrl: 905
      vol_head:
        ref: 0
        x:
          - 0
          - 39
          - 41.66
        y:
          - 860
          - 906
          - 907
      energy_value_input: 30
    Reservoir2:
      start_head: 670
      max_vol: 97.5
      lrl: 650
      hrl: 679
      vol_head:
        ref: 0
        x:
          - 0
          - 97.5
          - 104.15
        y:
          - 650
          - 679
          - 680
      inflow:
        2018-01-23 00:00:00: 60
      energy_value_input: 10
  plant:
    Plant1:
      equal_distribution: -1
      outlet_line: 672
      main_loss:
        - 0
      penstock_loss:
        - 0.001
      mip_flag:
        2018-01-23 00:00:00: 1
    Plant2:
      equal_distribution: -1
      outlet_line: 586
      main_loss:
    

## reserve_obligation.yaml <a name="reserve_obligation.yaml"></a>

In [24]:
%pycat reserve_obligation.yaml

model:
  reserve_group:
    fcr_n_group:
      fcr_n_up_obligation:
        2018-01-23 00:00:00: 10
      fcr_n_down_obligation:
        2018-01-23 00:00:00: 10
    frr_group:
      frr_up_obligation:
        2018-01-23 00:00:00: 15
connections:
  - from: fcr_n_group
    to: Plant1_G1
    from_type: reserve_group
    to_type: generator
    connection_type: connection_standard
    order: 0
  - from: fcr_n_group
    to: Plant1_G2
    from_type: reserve_group
    to_type: generator
    connection_type: connection_standard
    order: 0
  - from: fcr_n_group
    to: Plant2_G1
    from_type: reserve_group
    to_type: generator
    connection_type: connection_standard
    order: 0
  - from: frr_group
    to: Plant2_G1
    from_type: reserve_group
    to_type: generator
    connection_type: connection_standard
    order: 1
  - from: fcr_n_group
    to: Plant2_G2
    from_type: reserve_group
    to_type: generator
    connection_type: connection_standard
    order: 0
  - from: frr_group
    to

## smooth_reserve.yaml <a name="smooth_reserve.yaml"></a>

In [25]:
%pycat smooth_reserve.yaml

model:
  generator:
    Plant1_G1:
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
    Plant1_G2:
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
    Plant2_G1:
      frr_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
    Plant2_G2:
      frr_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
    Plant2_G3:
      frr_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
    Plant2_G4:
      frr_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_up_min:
        2018-01-23 00:00:00: 1
      fcr_n_down_min:
        2018-01-23 00:00:00: 1
commands:
    - set reserve_ramping_cost 5


## model.ascii <a name="model.ascii"></a>

In [26]:
%pycat model.ascii

#;
# Name of the datafile is: model.ascii;
#;

SIZE
#Num_reservoirs;Num_plants;Num_gates;Num_junctions
2            2          0           0

OPTIMIZATION time
#Start_time;   End_time;
20180123000000 20180126000000

#;N_full_iterations;Accuracy;
OPTIMIZATION   1000       1.00
#Time resolution in the optimization;
#Id;Number;Start_Time;Time_unit;Period;Data_type;Y_unit;Pts;
0      0 20180123000000000 HOUR      0     -1 NO_UNIT     73
#            Time;      f(t);
20180123000000000 1.00000000000000000

GLOBAL_SETTINGS mipgap_rel
# relative mipgap for the optimization;
0.000000

GLOBAL_SETTINGS mipgap_abs
# absolute mipgap_unit for the optimization;
0.000000

RESERVOIR attributes Reservoir1
#ID;Water_course;Type;Maxvol;Lrl;Hrl;
0      0      0     39.000    860.000    905.000

RESERVOIR vol_head Reservoir1
#Id;Number;Reference;Pts;X_unit;Y_unit
0      0       0.00      3 MM3 METER
#  x_value;   y_value;
0.0000000000 860.0000000000
39.0000000000 906.0000000000
41.6600000000 907.0000000000


## reserve_obligation.ascii <a name="reserve_obligation.ascii"></a>

In [27]:
%pycat reserve_obligation.ascii

RESERVE_GROUP   fcr_n_up        1
#Id;Number;Start_Time;Time_unit;Period;Data_type;Y_unit;Pts;
0      0 20180123000000000 HOUR      0     -1 MW     1
#            Time;      f(t);
20180123000000000 10.0

RESERVE_GROUP   fcr_n_down      1
#Id;Number;Start_Time;Time_unit;Period;Data_type;Y_unit;Pts;
0      0 20180123000000000 HOUR      0     -1 MW     1
#            Time;      f(t);
20180123000000000 10.0

RESERVE_GROUP   frr_up  2
#Id;Number;Start_Time;Time_unit;Period;Data_type;Y_unit;Pts;
0      0 20180123000000000 HOUR      0     -1 MW     1
#            Time;      f(t);
20180123000000000 15.0

#FCR-N UP
GENERATOR       gen_fcr_n_up_group      Plant1      1
#       Id      number  starttime               time_unit       period  data_type       y_unit  npts
0       0       20190913060000000       HOUR            8760            -1      NO_UNIT 1
#       time            y
20190913060000000       1.000000000000

GENERATOR       gen_fcr_n_up_group      Plant1      2
#       Id      numbe

## smooth_reserve.ascii <a name="smooth_reserve.ascii"></a>

In [28]:
%pycat smooth_reserve.ascii

GENERATOR       gen_fcr_n_up_min        Plant1      1
#       Id      number  starttime               time_unit       period  data_type       y_unit  npts
0       0       20190913060000000       HOUR            8760            -1      MW      1
#       time            y
20190913060000000       1.000000000000

GENERATOR       gen_fcr_n_up_min        Plant1      2
#       Id      number  starttime               time_unit       period  data_type       y_unit  npts
0       0       20190913060000000       HOUR            8760            -1      MW      1
#       time            y
20190913060000000       1.000000000000

GENERATOR       gen_fcr_n_up_min        Plant2      1
#       Id      number  starttime               time_unit       period  data_type       y_unit  npts
0       0       20190913060000000       HOUR            8760            -1      NO_UNIT 1
#       time            y
20190913060000000       1.000000000000

GENERATOR       gen_fcr_n_up_min        Plant2      2
#       Id   